In [15]:
import echr_extractor as echr
import re

dataframe, json = echr.get_echr_extra(count = 10, save_file = 'y', threads = 10, language = ["ENG"])

# Italian - ITA, Bulgarian - BUL, Polish - POL, Dutch - DUT, Croatian - HRV

txt: str = json[0]["full_text"]
txt = txt.replace(u"\xa0", u"\n")
txt = re.sub(r"\n{2,}", "\n", txt)

print(txt)

INFO:root:--- STARTING ECHR DOWNLOAD FOR  ---
INFO:root:http://hudoc.echr.coe.int/app/query/results?query=(contentsitename=ECHR)%20AND%20(documentcollectionid2:%22JUDGMENTS%22%20OR%20documentcollectionid2:%22COMMUNICATEDCASES%22%20OR%20documentcollectionid2:%22DECISIONS%22%20OR%20documentcollectionid2:%22CLIN%22)%20AND%20((languageisocode="ENG") OR (languageisocode:"ENG"))&select=itemid,applicability,appno,article,conclusion,docname,doctype,doctypebranch,ecli,importance,judgementdate,languageisocode,originatingbody,violation,nonviolation,extractedappno,scl,publishedby,representedby,respondent,separateopinion,sharepointid,externalsources,issue,referencedate,rulesofcourt,DocId,WorkId,Rank,Author,Size,Path,Description,Write,CollapsingStatus,HighlightedSummary,HighlightedProperties,contentclass,PictureThumbnailURL,ServerRedirectedURL,ServerRedirectedEmbedURL,ServerRedirectedPreviewURL,FileExtension,ContentTypeId,ParentLink,ViewsLifeTime,ViewsRecent,SectionNames,SectionIndexes,SiteLogo,Site


 In the case of Arrigoni v. Italy (1)_
 The Screening Panel of the European Court of Human Rights_constituted in accordance with Article 48 para. 2 (art. 48-2) of theConvention for the Protection of Human Rights and Fundamental Freedoms("the Convention") and Rule 26 of Rules of Court B (2)________________Notes by the Registrar
1.
 The case is numbered 26/1995/532/618.
 The first number is thecase's position on the list of cases referred to the Court in therelevant year (second number).
 The last two numbers indicate thecase's position on the list of cases referred to the Court since itscreation and on the list of the corresponding originating applicationsto the Commission.
2.
 Rules of Court B_ which came into force on 2 October 1994_ applyto all cases concerning the States bound by Protocol No. 9 (P9)._______________
 Sitting in private at Strasbourg on 28 April_ 29 June and1 September 1995_ and composed of the following judges:
 Mr Thór Vilhjálmsson_ Chairman_
 Mr F. Gölcüklü_
 Mr C

Basic (?) case structure

I. setup/introduction\
II. relevant legal framework\
III. court decision

data set from Ilias Chalkidis, Ion Androutsopoulos, and Nikolaos Aletras. 2019. Neural Legal Judgment Prediction in English. In Proceedings of the 57th Annual Meeting of the Association for Computational Linguistics, pages 4317–4323, Florence, Italy. Association for Computational Linguistics.

In [1]:
import json

# txt = json.load(open("DataPaper/EN_train/001-100005.json", "rb"))
# txt

from datasets import load_dataset

dataset = load_dataset("lex_glue", "ecthr_a")
txt = dataset["train"]["text"][0][0]
print(txt)

Using the latest cached version of the module from /home/keddie/.cache/huggingface/modules/datasets_modules/datasets/lex_glue/8a66420941bf6e77a7ddd4da4d3bfb7ba88ef48c1d55302a568ac650a095ca3a (last modified on Mon Nov 20 09:13:59 2023) since it couldn't be found locally at lex_glue., or remotely on the Hugging Face Hub.


11.  At the beginning of the events relevant to the application, K. had a daughter, P., and a son, M., born in 1986 and 1988 respectively. P.’s father is X and M.’s father is V. From March to May 1989 K. was voluntarily hospitalised for about three months, having been diagnosed as suffering from schizophrenia. From August to November 1989 and from December 1989 to March 1990, she was again hospitalised for periods of about three months on account of this illness. In 1991 she was hospitalised for less than a week, diagnosed as suffering from an atypical and undefinable psychosis. It appears that social welfare and health authorities have been in contact with the family since 1989.


In [59]:
dataset["train"]["text"][0]

['11.  At the beginning of the events relevant to the application, K. had a daughter, P., and a son, M., born in 1986 and 1988 respectively. P.’s father is X and M.’s father is V. From March to May 1989 K. was voluntarily hospitalised for about three months, having been diagnosed as suffering from schizophrenia. From August to November 1989 and from December 1989 to March 1990, she was again hospitalised for periods of about three months on account of this illness. In 1991 she was hospitalised for less than a week, diagnosed as suffering from an atypical and undefinable psychosis. It appears that social welfare and health authorities have been in contact with the family since 1989.',
 '12.  The applicants initially cohabited from the summer of 1991 to July 1993. In 1991 both P. and M. were living with them. From 1991 to 1993 K. and X were involved in a custody and access dispute concerning P. In May 1992 a residence order was made transferring custody of P. to X.',
 '13.  K. was again 

In [55]:
def preprocess_text(sentence):
    sentence = re.sub(r"[\n\t]+", " ", sentence)
    sentence = re.sub(r"[()\[\]]+", "", sentence)
    sentence = sentence.lower()
    sentence = re.sub(r"[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}”“‘’~–…]+", "", sentence)
    sentence = [token for token in sentence.split(" ") if len(token) > 0 and token.isalpha()]
    return sentence

In [66]:
from collections import defaultdict
import numpy as np

def encode_bow(training_data):
    vocabulary = defaultdict(int)

    sentence = preprocess_text(training_data)
    print(sentence)

    for token in sentence:
        vocabulary[token] += 1

    vocabulary = dict(sorted(vocabulary.items())) # avoid situations such as [1 1 1 1 0 0 ...]
    print(vocabulary)
    
    return np.asarray([int(vocab_token in sentence) for vocab_token in vocabulary.keys()])

vec = encode_bow("\n".join(dataset["train"]["text"][0]))
print(vec.shape)

['at', 'the', 'beginning', 'of', 'the', 'events', 'relevant', 'to', 'the', 'application', 'k', 'had', 'a', 'daughter', 'p', 'and', 'a', 'son', 'm', 'born', 'in', 'and', 'respectively', 'ps', 'father', 'is', 'x', 'and', 'ms', 'father', 'is', 'v', 'from', 'march', 'to', 'may', 'k', 'was', 'voluntarily', 'hospitalised', 'for', 'about', 'three', 'months', 'having', 'been', 'diagnosed', 'as', 'suffering', 'from', 'schizophrenia', 'from', 'august', 'to', 'november', 'and', 'from', 'december', 'to', 'march', 'she', 'was', 'again', 'hospitalised', 'for', 'periods', 'of', 'about', 'three', 'months', 'on', 'account', 'of', 'this', 'illness', 'in', 'she', 'was', 'hospitalised', 'for', 'less', 'than', 'a', 'week', 'diagnosed', 'as', 'suffering', 'from', 'an', 'atypical', 'and', 'undefinable', 'psychosis', 'it', 'appears', 'that', 'social', 'welfare', 'and', 'health', 'authorities', 'have', 'been', 'in', 'contact', 'with', 'the', 'family', 'since', 'the', 'applicants', 'initially', 'cohabited', 'fr